In [50]:
from PIL import Image, ImageFont, ImageDraw 
import glob
import string
import random
import os
import shutil
import math
import tensorflow as tf
print(tf.__version__)
from sklearn.utils import shuffle
from tensorflow import keras
from tensorflow.keras.layers import MaxPooling2D, BatchNormalization, Conv2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, Rescaling
from tensorflow.keras import Input 
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import seaborn as sns
import numpy as np 
import matplotlib.pyplot as plt 
import cv2

width = 128
height = 128

2.9.0


In [51]:
data_path = "C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data"
img_ext = ["jpg","png", "jpeg"];
sound_ext = ["mp3"];
ext = img_ext + sound_ext


def read_img(path):
    img = cv2.resize(cv2.imread(path, cv2.IMREAD_COLOR), (width, height))
    img = np.array(img) / 256.0
    return img

fr_model = keras.models.load_model('Face_recognition.h5')

In [52]:
imgs = []
images = []
labels = []
sounds = []

for path in glob.glob(f'{data_path + "/True"}/*/'):
    for files in (files for files in (glob.glob(path + '/*.%s' % e) for e in ext) if files != []):
        if (os.path.splitext(files[0])[1].split(".")[1] in sound_ext):
             sounds.append(files[0])
        else:
            for file in files:
                imgs.append(read_img(file))
            images.append(fr_model.predict(np.array(imgs), verbose = 0))
            labels.append(1.0)
            imgs = []
        
for path in glob.glob(f'{data_path + "/False"}/*/'):
    for files in (files for files in (glob.glob(path + '/*.%s' % e) for e in ext) if files != []):
        if (os.path.splitext(files[0])[1].split(".")[1] in sound_ext):
             sounds.append(files[0])
        else:
            for file in files:
                imgs.append(read_img(file))
            images.append(fr_model.predict(np.array(imgs), verbose = 0))
            labels.append(0.0)
            imgs = []

print("Imgs len: ", len(imgs))
print("Labels len: ", len(labels))
print("Images len: ", len(images))
print("Sounds len: ", len(sounds))

print("Images arr len: ", len(images[0]))

Imgs len:  0
Labels len:  17
Images len:  17
Sounds len:  17
Images arr len:  30


In [53]:
print(images[0])

[[0.61297554]
 [0.6129929 ]
 [0.612984  ]
 [0.61302054]
 [0.6141629 ]
 [0.61777323]
 [0.620445  ]
 [0.62266374]
 [0.620423  ]
 [0.62128925]
 [0.62037486]
 [0.6090984 ]
 [0.62113464]
 [0.62089354]
 [0.61971855]
 [0.6197652 ]
 [0.62283236]
 [0.619907  ]
 [0.619413  ]
 [0.61504716]
 [0.61329   ]
 [0.61735857]
 [0.6114036 ]
 [0.61929506]
 [0.6134375 ]
 [0.6114385 ]
 [0.6120786 ]
 [0.6106381 ]
 [0.6122975 ]
 [0.6127975 ]]
